In [1]:
from langsmith import Client
from langsmith.evaluation import evaluate
from langsmith.schemas import Example, Run

# Initialize LangSmith client
client = Client()

In [ ]:
dataset = client.create_dataset(
    "Customer Support RAG Dataset",
    description="A dataset for evaluating RAG performance in customer support scenarios"
)

In [ ]:
client.create_examples(
    inputs=[
        {"input": "¿Cuáles son los canales de atención al cliente que ofrece TechnoVerde?"},
        {"input": "¿Cuál es el tiempo de respuesta objetivo para los correos electrónicos?"},
        {"input": "¿Cuánto tiempo tengo para devolver un producto?"},
        {"input": "¿Qué cubre la garantía de los productos TechnoVerde?"},
        {"input": "¿Cómo funciona el programa de lealtad de TechnoVerde?"},
        {"input": "¿Qué medidas toma TechnoVerde para proteger los datos de los clientes?"},
        {"input": "¿Qué tipo de formación reciben los empleados de atención al cliente?"},
        {"input": "¿Cómo maneja TechnoVerde las situaciones de crisis?"},
        {"input": "¿Qué tecnologías innovadoras utiliza TechnoVerde en su atención al cliente?"},
        {"input": "¿Cómo asegura TechnoVerde la accesibilidad en su atención al cliente?"}
    ],
    outputs=[
        {"response": "TechnoVerde S.A. ofrece múltiples canales de atención al cliente: una línea telefónica gratuita (0800-TECNO-VERDE) disponible 24/7, correo electrónico (soporte@technoverde.com), chat en vivo en su sitio web (www.technoverde.com), redes sociales (@TechnoVerdeOficial en Facebook, Twitter e Instagram), y atención presencial en sus tiendas de lunes a sábado de 9:00 a 20:00."},
        {"response": "Según la política de atención al cliente de TechnoVerde, el tiempo de respuesta objetivo para los correos electrónicos es de menos de 4 horas hábiles."},
        {"response": "De acuerdo con la política de reembolsos y devoluciones de TechnoVerde, los clientes pueden devolver productos no utilizados dentro de los 30 días posteriores a la compra para obtener un reembolso completo o un cambio."},
        {"response": "Todos los productos TechnoVerde están cubiertos por una garantía mínima de 2 años. Algunos productos premium tienen garantías extendidas de hasta 5 años. Además, la empresa ofrece soporte técnico gratuito durante toda la vida útil del producto."},
        {"response": "El programa de lealtad de TechnoVerde funciona de la siguiente manera: los clientes son automáticamente inscritos al realizar su primera compra, ganan 1 punto por cada $10 gastados, y hay cuatro niveles de membresía (Verde, Plata, Oro y Platino) con beneficios crecientes. Los beneficios incluyen acceso anticipado a nuevos productos, invitaciones a eventos exclusivos y descuentos especiales."},
        {"response": "TechnoVerde toma varias medidas para proteger los datos de los clientes. Solo recopilan los datos necesarios para proporcionar sus servicios y mejorar la experiencia del cliente. Todos los datos se almacenan de forma segura en servidores encriptados y se implementan medidas de seguridad de vanguardia para proteger la información contra accesos no autorizados."},
        {"response": "Los empleados de atención al cliente de TechnoVerde reciben un programa de formación inicial intensivo de 4 semanas que cubre conocimiento de productos y servicios, habilidades de comunicación y resolución de problemas, políticas y procedimientos de la empresa, y uso de sistemas de atención al cliente. Además, se ofrecen programas de formación continua mensuales para mantenerlos actualizados."},
        {"response": "TechnoVerde mantiene un plan detallado de continuidad del negocio para garantizar la atención al cliente en casos de crisis como desastres naturales, fallos tecnológicos o pandemias. En situaciones de crisis, se comprometen a informar proactivamente a los clientes sobre cualquier interrupción del servicio, proporcionar actualizaciones regulares y ofrecer soluciones alternativas cuando sea posible."},
        {"response": "TechnoVerde utiliza varias tecnologías innovadoras en su atención al cliente, incluyendo soluciones de Inteligencia Artificial y chatbots para proporcionar respuestas instantáneas y analizar el sentimiento del cliente. También implementan tecnología de Realidad Aumentada (RA) para guiar a los clientes en la instalación y resolución de problemas de productos, así como para proporcionar demostraciones virtuales."},
        {"response": "TechnoVerde asegura la accesibilidad en su atención al cliente mediante un enfoque de diseño universal. Esto incluye un sitio web y aplicaciones móviles compatibles con lectores de pantalla, opciones de comunicación para clientes con discapacidad auditiva o del habla, y materiales de soporte en múltiples idiomas y formatos. Además, todo el personal de atención al cliente recibe formación regular sobre diversidad e inclusión."}
    ],
    dataset_id=dataset.id
)

In [3]:
#AI App

from agent import Agent
from langchain_chroma import Chroma
from tools import create_retriever_tool_from_vectorstore
from langchain_core.messages import HumanMessage
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import AIMessage
persist_directory = "./chroma_db"
vectorstore = Chroma(
    collection_name="rag-chroma",
    embedding_function=OpenAIEmbeddings(),
    persist_directory=persist_directory
)

tools = [create_retriever_tool_from_vectorstore(vectorstore)]


def start_conversation(input_dict):
    try:
        agent = Agent(model_type="groq", tools=tools)
        human_message = HumanMessage(content=input_dict['input'])
        state = agent.invoke([human_message])
        
        if isinstance(state, list) and len(state) > 0 and isinstance(state[-1], AIMessage):
            return {"answer": state[-1].content}
        else:
            return {"answer": "No AI response found in the state."}
    except Exception as e:
        return {"error": str(e)}

# Example usage
result = start_conversation({'input': 'Buenas tardes'})
result

{'answer': 'Buenas tardes. ¿En qué puedo ayudarte hoy?'}

In [4]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

answer_correctness_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "answer_correctness": """Is the Assistant's Answer aligned with the Ground Truth Answer? A score of [[1]] means that the
            Assistant answer contains is not at all based upon / grounded in the Groun Truth Answer. A score of [[5]] means 
            that the Assistant answer contains some information (e.g., a hallucination) that is not captured in the Ground Truth 
            Answer. A score of [[10]] means that the Assistant answer is fully based upon the in the Ground Truth Answer."""
        },
        # If you want the score to be saved on a scale from 0 to 1
        "normalize_by": 10,
        "key": "answer_correctness"
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": example.outputs["response"],
        "input": example.inputs["input"],
    },
)

dataset_name = "Customer Support RAG Dataset"
experiment_results = evaluate(
    start_conversation,
    data=dataset_name,
    evaluators=[answer_correctness_evaluator],
    experiment_prefix="rag-answer-correctness-llama",

)

View the evaluation results for experiment: 'rag-answer-correctness-llama-621dcdd1' at:
https://smith.langchain.com/o/e78e2c5a-05c5-5edd-a70a-d8f4dd524804/datasets/46e63ba0-3af3-4fbb-98b8-4c7555c9968c/compare?selectedSessions=425c9665-9ae7-46f5-9394-ffb3b2fc22fc




1it [00:23, 23.76s/it]Error running evaluator <DynamicRunEvaluator evaluate> on run 6116b112-d256-4411-b92d-32331e380dc0: KeyError('answer')
Traceback (most recent call last):
  File "/Users/lucaspetralli/Documents/LangGraph-easy-template/easy-lang-env/lib/python3.11/site-packages/langsmith/evaluation/_runner.py", line 1321, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/lucaspetralli/Documents/LangGraph-easy-template/easy-lang-env/lib/python3.11/site-packages/langsmith/evaluation/evaluator.py", line 278, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "/Users/lucaspetralli/Documents/LangGraph-easy-template/easy-lang-env/lib/python3.11/site-packages/langsmith/run_helpers.py", line 609, in wrapper
    raise e
  File "/Users/lucaspetralli/Documents/LangGraph-easy-template/easy-lang-env/lib/python3.11/site-packages/langsmith/run_helpers.py", line 606, in wrapper
    function_result 